<a href="https://colab.research.google.com/github/vthylur/Backtesting/blob/NSE-data-extract/NSE_data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# === STEP 1: Install Dependencies ===
!pip install yfinance ta openpyxl --quiet

# === STEP 2: Import Libraries ===
import yfinance as yf
import pandas as pd
import numpy as np
import ta
from datetime import datetime
from zipfile import ZipFile
import os
import time

# === STEP 3: Indicator Calculation ===
def calculate_indicators(df, close_col, high_col, low_col, volume_col, time_period_label):
    df_copy = df.copy()

    # Bollinger Bands
    bb_window = 20
    bb_std_dev = 2
    df_copy['BB_SMA'] = df_copy[close_col].rolling(window=bb_window).mean()
    df_copy['STD'] = df_copy[close_col].rolling(window=bb_window).std()
    df_copy['BB_upper'] = df_copy['BB_SMA'] + bb_std_dev * df_copy['STD']
    df_copy['BB_lower'] = df_copy['BB_SMA'] - bb_std_dev * df_copy['STD']
    df_copy['Close_vs_BB'] = df_copy[close_col] / df_copy['BB_upper'].replace(0, np.nan)
    df_copy['BB_width'] = df_copy['BB_upper'] - df_copy['BB_lower']
    df_copy['BB_Squeeze'] = df_copy['BB_width'] < df_copy['BB_width'].quantile(0.2)

    # Keltner Channel + TTM Squeeze
    ema = ta.trend.ema_indicator(df_copy[close_col], window=20)
    atr = ta.volatility.average_true_range(df_copy[high_col], df_copy[low_col], df_copy[close_col], window=20)
    df_copy['KC_upper'] = ema + 1.5 * atr
    df_copy['KC_lower'] = ema - 1.5 * atr
    df_copy['TTM_Squeeze_On'] = (df_copy['BB_upper'] < df_copy['KC_upper']) & (df_copy['BB_lower'] > df_copy['KC_lower'])
    df_copy['TTM_Squeeze_Off'] = (df_copy['BB_upper'] > df_copy['KC_upper']) & (df_copy['BB_lower'] < df_copy['KC_lower'])

    # Indicators
    df_copy['MACD'] = ta.trend.macd(df_copy[close_col])
    df_copy['RSI'] = ta.momentum.rsi(df_copy[close_col])
    df_copy['ROC'] = ta.momentum.roc(df_copy[close_col])
    df_copy['OBV'] = ta.volume.on_balance_volume(df_copy[close_col], df_copy[volume_col])
    df_copy['VWAP'] = ((df_copy[high_col] + df_copy[low_col] + df_copy[close_col]) / 3 * df_copy[volume_col]).cumsum() / df_copy[volume_col].cumsum().replace(0, np.nan)

    # ADX and Stochastic
    adx = ta.trend.ADXIndicator(df_copy[high_col], df_copy[low_col], df_copy[close_col])
    df_copy['ADX'] = adx.adx()
    df_copy['+DI'] = adx.adx_pos()
    df_copy['-DI'] = adx.adx_neg()

    stoch = ta.momentum.StochasticOscillator(df_copy[high_col], df_copy[low_col], df_copy[close_col])
    df_copy['Stoch_k'] = stoch.stoch()
    df_copy['Stoch_d'] = stoch.stoch_signal()

    # SMA & EMA
    for w in [20, 50, 200]:
        df_copy[f'SMA_{w}{time_period_label}'] = ta.trend.sma_indicator(df_copy[close_col], window=w)
        df_copy[f'EMA_{w}{time_period_label}'] = ta.trend.ema_indicator(df_copy[close_col], window=w)

    # Support / Resistance
    df_copy['Support'] = df_copy[close_col].rolling(window=20).min()
    df_copy['Resistance'] = df_copy[close_col].rolling(window=20).max()

    # Fibonacci (safe fallback for small dataframes)
    if len(df_copy) >= 20:
        recent = df_copy.tail(100)
        hi = recent[high_col].max()
        lo = recent[low_col].min()
        rng = hi - lo
        df_copy['Fib_23.6'] = hi - 0.236 * rng
        df_copy['Fib_38.2'] = hi - 0.382 * rng
        df_copy['Fib_50.0'] = hi - 0.500 * rng
        df_copy['Fib_61.8'] = hi - 0.618 * rng
    else:
        df_copy['Fib_23.6'] = np.nan
        df_copy['Fib_38.2'] = np.nan
        df_copy['Fib_50.0'] = np.nan
        df_copy['Fib_61.8'] = np.nan

    # Volume MAs
    for w in [20, 50]:
        df_copy[f'Vol_SMA_{w}{time_period_label}'] = df_copy[volume_col].rolling(window=w).mean()
    df_copy[f'Vol_vs_SMA20{time_period_label}'] = df_copy[volume_col] / df_copy[f'Vol_SMA_20{time_period_label}'] - 1

    df_copy.drop(columns=['BB_SMA', 'STD'], errors='ignore', inplace=True)
    return df_copy

# === STEP 4: Run in Batches ===
def chunk_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

symbols = ["360ONE.NS", "3MINDIA.NS", "ABB.NS", "ACC.NS", "ACMESOLAR.NS", "AIAENG.NS", "APLAPOLLO.NS", "AUBANK.NS", "AWL.NS", "AADHARHFC.NS", "AARTIIND.NS", "AAVAS.NS", "ABBOTINDIA.NS", "ACE.NS", "ADANIENSOL.NS", "ADANIENT.NS", "ADANIGREEN.NS", "ADANIPORTS.NS", "ADANIPOWER.NS", "ATGL.NS", "ABCAPITAL.NS", "ABFRL.NS", "ABLBL.NS", "ABREL.NS", "ABSLAMC.NS", "AEGISLOG.NS", "AFCONS.NS", "AFFLE.NS", "AJANTPHARM.NS", "AKUMS.NS", "APLLTD.NS", "ALIVUS.NS", "ALKEM.NS", "ALKYLAMINE.NS", "ALOKINDS.NS", "ARE&M.NS", "AMBER.NS", "AMBUJACEM.NS", "ANANDRATHI.NS", "ANANTRAJ.NS", "ANGELONE.NS", "APARINDS.NS", "APOLLOHOSP.NS", "APOLLOTYRE.NS", "APTUS.NS", "ASAHIINDIA.NS", "ASHOKLEY.NS", "ASIANPAINT.NS", "ASTERDM.NS", "ASTRAZEN.NS", "ASTRAL.NS", "ATUL.NS", "AUROPHARMA.NS", "AIIL.NS"]  # Replace with full list
start_date = "2003-01-01"
output_folder = "stocks_output"
os.makedirs(output_folder, exist_ok=True)
batch_size = 20
delay_between_batches = 10
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

for batch_index, batch in enumerate(chunk_list(symbols, batch_size), start=1):
    print(f"\n🚀 Starting batch {batch_index} with {len(batch)} symbols")
    for symbol in batch:
        try:
            df = yf.download(symbol, start=start_date, interval="1d", auto_adjust=False, progress=False)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)
            df.columns = [col.capitalize() for col in df.columns]
            if df.empty or not all(c in df.columns for c in ['Open', 'High', 'Low', 'Close', 'Volume']):
                print(f"⚠️ Skipping {symbol} (incomplete data)")
                continue
            df.index = pd.to_datetime(df.index)

            df_daily = calculate_indicators(df.copy(), 'Close', 'High', 'Low', 'Volume', 'd')
            df_daily.reset_index(inplace=True)
            df_daily.rename(columns={'index': 'Date'}, inplace=True)

            df_weekly = calculate_indicators(df.resample('W').agg({
                'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'
            }).dropna(), 'Close', 'High', 'Low', 'Volume', 'w')
            df_weekly.reset_index(inplace=True)
            df_weekly.rename(columns={'index': 'Date'}, inplace=True)

            df_monthly = calculate_indicators(df.resample('ME').agg({
                'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'
            }).dropna(), 'Close', 'High', 'Low', 'Volume', 'm')
            df_monthly.reset_index(inplace=True)
            df_monthly.rename(columns={'index': 'Date'}, inplace=True)

            output_file = f"{output_folder}/{symbol.replace('.NS','')}_{timestamp_str}.xlsx"
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                df_daily.to_excel(writer, sheet_name="Daily", index=False)
                df_weekly.to_excel(writer, sheet_name="Weekly", index=False)
                df_monthly.to_excel(writer, sheet_name="Monthly", index=False)
            print(f"✅ {symbol} saved")

        except Exception as e:
            print(f"❌ Error processing {symbol}: {e}")

    print(f"✅ Finished batch {batch_index}. Waiting {delay_between_batches}s before next batch...\n")
    time.sleep(delay_between_batches)

# === STEP 5: Zip Output ===
zip_path = f"{output_folder}.zip"
with ZipFile(zip_path, 'w') as zipf:
    for root, _, files in os.walk(output_folder):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)

from IPython.display import FileLink
FileLink(zip_path)



🚀 Starting batch 1 with 20 symbols
✅ 360ONE.NS saved
✅ 3MINDIA.NS saved
✅ ABB.NS saved
✅ ACC.NS saved
❌ Error processing ACMESOLAR.NS: index 19 is out of bounds for axis 0 with size 8
✅ AIAENG.NS saved
✅ APLAPOLLO.NS saved
✅ AUBANK.NS saved
✅ AWL.NS saved
❌ Error processing AADHARHFC.NS: index 19 is out of bounds for axis 0 with size 14
✅ AARTIIND.NS saved
✅ AAVAS.NS saved
✅ ABBOTINDIA.NS saved
✅ ACE.NS saved
❌ Error processing ADANIENSOL.NS: index 14 is out of bounds for axis 0 with size 10
✅ ADANIENT.NS saved
✅ ADANIGREEN.NS saved
✅ ADANIPORTS.NS saved
✅ ADANIPOWER.NS saved
✅ ATGL.NS saved
✅ Finished batch 1. Waiting 10s before next batch...


🚀 Starting batch 2 with 20 symbols
✅ ABCAPITAL.NS saved
✅ ABFRL.NS saved
❌ Error processing ABLBL.NS: index 19 is out of bounds for axis 0 with size 3
✅ ABREL.NS saved
✅ ABSLAMC.NS saved
❌ Error processing AEGISLOG.NS: index 19 is out of bounds for axis 0 with size 14
❌ Error processing AFCONS.NS: index 19 is out of bounds for axis 0 with size

/content/stocks_output.zip